In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv
from sklearn.utils import shuffle


%matplotlib inline



%load_ext autoreload
%autoreload 2


In [2]:
mnist = pd.read_csv('/home/oyku/Desktop/Oyku/Convex 10-725/train_mnist.csv')
mnist = pd.DataFrame.as_matrix(mnist)

mnist_shuffled = shuffle(mnist)

print mnist.shape

(42000, 785)


In [3]:
label = mnist[:,0]
data = mnist[:,1::]
print np.sum((label == 0) + 0)
print np.sum((label == 1) + 0)

new_data = mnist[(label < 2), :]
new_label = label[(label < 2)]
print new_data.shape
print new_label.shape

4132
4684
(8816, 785)
(8816,)


In [4]:
X = new_data
y = new_label.reshape([new_label.shape[0], 1])
y[y == 0] = -1
print X.shape
print y.shape

(8816, 785)
(8816, 1)


In [5]:
#Centralize the data
for i in xrange(X.shape[1]):
    X[:,i] = X[:,i] - np.mean(X[:,i])
X = np.append(X, np.ones([X.shape[0],1]), axis = 1)
print X.shape

(8816, 786)


In [15]:
print 6216 + 3108



9324


In [13]:
#Train and Validation sets
train = X[0:6216,:]
train_y = y[0:6216,:]

train = X[6216:9324,:]
train_y = y[6216:9324,:]

train = X[9324:,:]
train_y = y[0:6216,:]

X, [X.shape[0]-2600, 3108, 3108], axis = 0)
print train.shape
print valid.shape
print test.shape




ValueError: too many values to unpack

In [6]:
#model parameters
n = X.shape[0] #number of examples in train
d = X.shape[1] #number of features in train

C = 10
epsilon = 0.5
t = 5

beta = 0.9 #backtracking step decrease rate
t_mu = 1.2 #scaling factor of t for the barrier method

In [7]:
w = np.random.rand(1,d)*1e-5
xi = np.ones([n,1])
while np.sum((y*(w.dot(X.T).T) >= epsilon - xi) + 0) < n:
    xi = np.ones([n,1])
    w = np.random.rand(1,d)*1e-5
print "w and xi initialized according to constraints"

w and xi initialized according to constraints


In [ ]:
epoch = 25
loss = np.zeros([1,epoch])
accuracy = np.zeros([1,epoch])
for e in xrange(epoch):
    step = 1
    
    #gradients
    denom = y*(w.dot(X.T).T) - epsilon + xi
    dw = t*w - np.sum(1/(1.0*denom)* y.T.dot(X), 0)
    dxi = t*C - 1/(1.0*denom) - 1/(1.0*xi)
    dw /= n
    dxi /= n

    #gradient descent
    temp_w = w - step*dw
    temp_xi = xi - step*dxi

    count = 0
    #while constraints are not obeyed
    while np.sum((y*(temp_w.dot(X.T).T) >= epsilon - temp_xi) + 0) + np.sum((temp_xi >= 0) + 0) < 2*n: 
        step = step*beta
        temp_w = w - step*dw
        temp_xi = xi - step*dxi
        count += 1
    print "COUNT :", count
    denom_left = y*(temp_w.dot(X.T).T) - epsilon + temp_xi


    loss_right = t * (0.5*w[:,0:-1].dot(w[:,0:-1].T) + C * np.sum(xi))
    loss_right -= np.sum(np.log(denom)) + np.sum(np.log(xi)) #barrier method
    loss_right /= n

    loss_left = t * (0.5*temp_w[:,0:-1].dot(temp_w[:,0:-1].T) + C * np.sum(temp_xi))
    loss_left -= np.sum(np.log(denom_left)) + np.sum(np.log(temp_xi)) 
    loss_left /= n

    #Backtracking
    count1 = 0
    do = 0
    while do == 0:
        step = beta*step
        temp_w = w - step*dw
        temp_xi = xi - step*dxi

        loss_right = t * (0.5*w[:,0:-1].dot(w[:,0:-1].T) + C * np.sum(xi))
        loss_right -= np.sum(np.log(denom)) + np.sum(np.log(xi)) #barrier method
        loss_right /= n

        loss_left = t * (0.5*temp_w[:,0:-1].dot(temp_w[:,0:-1].T) + C * np.sum(temp_xi))
        loss_left -= np.sum(np.log(denom_left)) + np.sum(np.log(temp_xi)) 
        loss_left /= n
        count1 += 1
        
        if loss_left <= loss_right:
            do = 1
        if count1 > 500:
            do = 1
    print "COUNT 1 :", count1
    loss[:,e] = loss_left
    #finalize the gradient update
    w = temp_w
    xi = temp_xi

    #end of Backtracking


    t = t*t_mu
    

    predicted = (w.dot(X.T).T > 0) + 0
    predicted[predicted == 0] = -1
    accuracy[:,e] = np.sum((predicted - y == 0) + 0)/(1.0*n)*100
    print "-------- ", e, " --------"
    print "Learning rate from backtracking: ", step
    print "Loss: ", loss[:,e]
    print "Prediction accuracy: ",accuracy[:,e]


In [ ]:
#print loss[:,-1]
print accuracy[:,-1]

In [ ]:
print predicted


In [ ]:
print y

step = 1

epoch = 20
loss = np.zeros([1,epoch])
accuracy = np.zeros([1,epoch])
for e in xrange(epoch):
    denom = y*(w.dot(X.T).T) - epsilon + xi

    #gradients
    dw = t*w - np.sum(1/(1.0*denom)* y.T.dot(X), 0)
    dxi = t*C - 1/(1.0*denom) - 1/(1.0*xi)
    dw /= n
    dxi /= n

    #gradient descent
    w_left = w - step*dw
    xi_left = xi - step*dxi
    
    #f_left = t *(0.5*w[:,0:-1].dot(w[:,0:-1].T) + C * np.sum(xi)) 

    loss[:,e] = t * (0.5*w[:,0:-1].dot(w[:,0:-1].T) + C * np.sum(xi))
    loss[:,e] -= np.sum(np.log(denom)) + np.sum(np.log(xi)) #barrier method
    loss[:,e] /= n
    

    predicted = (w.dot(X.T).T > 0) + 0
    predicted[predicted == 0] = -1
    accuracy[:,e] = np.sum((predicted - y == 0) + 0)/(1.0*n)*100
    print "-------- ", e, " --------"
    print "Loss: ", loss[:,e]
    print "Prediction accuracy: ",accuracy[:,e]